In [1]:
import os
import gc
import numpy as np
import pandas as pd
from tqdm import tqdm
from typing import Dict, List, Tuple, NamedTuple, Callable
import scml
import mylib

In [2]:
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()

In [3]:
%%time
df = pd.read_csv("input/validation_data.csv", engine="c", low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30108 entries, 0 to 30107
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   worker      30108 non-null  int64 
 1   less_toxic  30108 non-null  object
 2   more_toxic  30108 non-null  object
dtypes: int64(1), object(2)
memory usage: 705.8+ KB
Wall time: 216 ms


In [4]:
texts = set(df["less_toxic"]) | set(df["more_toxic"])
texts = list(texts)
df = pd.DataFrame(data={"text": texts})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14251 entries, 0 to 14250
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    14251 non-null  object
dtypes: object(1)
memory usage: 111.5+ KB


# Preprocess Text
Speed dropped from 1200 to 1000 it/s

In [5]:
def pre1(row) -> str:
    return mylib.pre1(row["text"])


def pre2(row) -> str:
    return mylib.pre2(row["text1"])


col = "text1"
print(col)
df[col] = df.progress_apply(pre1, axis=1)
col = "text2"
print(col)
df[col] = df.progress_apply(pre2, axis=1)

text1


100%|████████████████████████████████████████| 14251/14251 [00:46<00:00, 307.12it/s]


text2


100%|█████████████████████████████████████████| 14251/14251 [04:15<00:00, 55.88it/s]


In [6]:
cols = ["text", "text1", "text2"]
df[cols].sample(10).head(10)

,text,text1,text2
6894,tide rolls is a fucking moron he needs to go to HELL,tide rolls is a fucking moron he needs to go to HELL,tide rolls is a fucking moron he needs to go to HELL
10884,"except the Gay Republican bit, which is made up)","except the Gay Republican bit, which is made up)","except the Gay Republican bit, which is made up)"
6684,that you are a homosexual who is very gay.,that you are a homosexual who is very gay.,that you are a homosexual who is very gay.
2132,"""Disgusting==\nThis guy is a disgusting sex columnist. He revels in it. He does it on purpose, and it's what gives him and his column their edge. It seperates him from every other sex advice giver I've ever read and it's his signature attitude, tone, and approach to the issues. But there seems to be no way to adduce this highly pertinent and relevant information without violating the lame, loose, sloppy, ill-thought-out NPOV rule. Hilariously (and disgustingly), people tell me providing this information violates article """"neutrality."""" Any help here by any person of quality, integrity, and intelligence would be appreciated. —Preceding unsigned comment added by 24.44.129.156 \nImagine that a sex columnist who enjoys sex. How can we work that into the article? ;-) \n\n==""","""Disgusting== This guy is a disgusting sex columnist. He revels in it. He does it on purpose, and it's what gives him and his column their edge. It seperates him from every other sex advice giver I've ever read and it's his signature attitude, tone, and approach to the issues. But there seems to be no way to adduce this highly pertinent and relevant information without violating the lame, loose, sloppy, ill-thought-out NPOV rule. Hilariously (and disgustingly), people tell me providing this information violates article """"neutrality."""" Any help here by any person of quality, integrity, and intelligence would be appreciated. - Preceding unsigned comment added by 24.44.129.156 Imagine that a sex columnist who enjoys sex. How can we work that into the article? [Wink or smirk] ==""","""Disgusting== This guy is a disgusting sex columnist. He revels in it. He does it on purpose, and it is what gives him and his column their edge. It seperates him from every other sex advice giver I have ever read and it is his signature attitude, tone, and approach to the issues. But there seems to be no way to adduce this highly pertinent and relevant information without violating the lame, loose, sloppy, ill-thought-out NPOV rule. Hilariously (and disgustingly), people tell me providing this information violates article """"neutrality."""" Any help here by any person of quality, integrity, and intelligence would be appreciated. - Preceding unsigned comment added by 24.44.129.156 Imagine that a sex columnist who enjoys sex. How can we work that into the article? [Wink or smirk] =="""
2475,"Your intelligence, insight, and manners are quite poor. And YOU'RE the vandal, obviously. Don't waste my time, loser.","Your intelligence, insight, and manners are quite poor. And YOU'RE the vandal, obviously. Don't waste my time, loser.","Your intelligence, insight, and manners are quite poor. And you are the vandal, obviously. do not waste my time, loser."
6598,"Her blatant bias needs to be mentioned in the article \n\nMegan Kelly is, without a doubt, the most vitriolic, pro-Republican, anti-anything/anyone-who-isn't-Republican female pundit on Fox News. It is sickening to me (and to most rational human beings) that she is even considered a journalist. Brit Hume is a journalist. Shepard Smith is a journalist. George Will is a journalist. Megyn Kelly is a Republican Kool Aid drinker with a microphone and a camera. Now, I'm anticipating that some pro-Megyn zealot will revert my edit that she is a conservative so please consider why I made the edit - because it's valid. And lest you think I'm some liberal Fox hater, I would also like to see Keith Olbermann be called a liberal once and for all on Wikipedia.","Her bl

In [7]:
df[cols].sample(10).head(10)

text  \
1508                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [8]:
df[cols].sample(10).head(10)

,text,text1,text2
12309,"other rules. I take allegations of homosexuality about English monarchs very seriously, but you don't care, do you?","other rules. I take allegations of homosexuality about English monarchs very seriously, but you don't care, do you?","other rules. I take allegations of homosexuality about English monarchs very seriously, but you do not care, do you?"
8312,I went ahead and shortened the stuff on abortion and euthanasia.,I went ahead and shortened the stuff on abortion and euthanasia.,I went ahead and shortened the stuff on abortion and euthanasia.
7584,Hello I am the one known as the vandal person...you may call me kanden7 pf from now on. I see you mean business ronbo ). Don't you think putting me on that page with the Mega vandals is a bit extreme???,Hello I am the one known as the vandal person...you may call me kanden7 pf from now on. I see you mean business ronbo). Don't you think putting me on that page with the Mega vandals is a bit extreme???,Hello I am the one known as the vandal person...you may call me kanden7 pf from now on. I see you mean business ronbo). do not you think putting me on that page with the Mega vandals is a bit extreme???
7656,"""\nI'm afraid I don't see the problem you seem to have. The table looks OK even at low (<900px wide) resolutions. Your comment that """"adding extra columns which have no content other than N/A is ridiculous as it shows absolutely no information of any kind"""" ignores the fact that there is content in these columns. Obviously there is presently none for seasons that haven't been released on Blu-ray but this is something that exists now in many tables for DVD entries. As Blu-ray versions are released, this information will be supplied. A separate Blu-ray table would unnecessarily duplicate content ad somebody would no doubt complain about that. An all inclusive table seems the best compromise. """,""" I'm afraid I don't see the problem you seem to have. The table looks OK even at low (<900px wide) resolutions. Your comment that """"adding extra columns which have no content other than N/A is ridiculous as it shows absolutely no information of any kind"""" ignores the fact that there is content in these columns. Obviously there is presently none for seasons that haven't been released on Blu-ray but this is something that exists now in many tables for DVD entries. As Blu-ray versions are released, this information will be supplied. A separate Blu-ray table would unnecessarily duplicate content ad somebody would no doubt complain about that. An all inclusive table seems the best compromise. """,""" I am afraid I do not see the problem you seem to have. The table looks OK even at low (<900px wide) resolutions. Your comment that """"adding extra columns which have no content other than N/A is ridiculous as it shows absolutely no information of any kind"""" ignores the fact that there is content in these columns. Obviously there is presently none for seasons that have not been released on Blu-ray but this is something that exists now in many tables for DVD entries. As Blu-ray versions are released, this information will be supplied. A separate Blu-ray table would unnecessarily duplicate content ad somebody would no doubt complain about that. An all inclusive table seems the best compromise. """
436,"Thomas Cannon \n\nLaughable commentary from another one of Wikipedia's ignorant, lazy, and arrogant. He deleted my reply on his own Talk page. But of course! He should consider himself lucky to get it. See Point 4 on my User Page","Thomas Cannon Laughable commentary from another one of Wikipedia's ignorant, lazy, and arrogant. He deleted my reply on his own Talk page. But of course! He should consider himself lucky to get it. See Point 4 on my User Page","Thomas Cannon Laughable commentary from another one of Wikipedia's ignorant, lazy, and arrogant. He deleted my reply on his own Talk page. But of course! He should consider himself lucky to get it. See Point 4 

# Review data

In [9]:
df[cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14251 entries, 0 to 14250
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    14251 non-null  object
 1   text1   14251 non-null  object
 2   text2   14251 non-null  object
dtypes: object(3)
memory usage: 334.1+ KB


In [10]:
%%time
df[cols].to_parquet("output/pre_val.parquet", index=False)

Wall time: 101 ms
